In [139]:
import helpers
import numpy as np
import math

data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""
with open("input_11.txt", 'r') as f:
    data = f.read()
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = [changeDict[x.split(" ")[0]] * int(x.split(" ")[1]) for x in data.split("\n") if x]
data = [x.split('\n') for x in data.split('\n\n') if x]
#np.prod(sum(data))    
data


[['Monkey 0:',
  '  Starting items: 72, 64, 51, 57, 93, 97, 68',
  '  Operation: new = old * 19',
  '  Test: divisible by 17',
  '    If true: throw to monkey 4',
  '    If false: throw to monkey 7'],
 ['Monkey 1:',
  '  Starting items: 62',
  '  Operation: new = old * 11',
  '  Test: divisible by 3',
  '    If true: throw to monkey 3',
  '    If false: throw to monkey 2'],
 ['Monkey 2:',
  '  Starting items: 57, 94, 69, 79, 72',
  '  Operation: new = old + 6',
  '  Test: divisible by 19',
  '    If true: throw to monkey 0',
  '    If false: throw to monkey 4'],
 ['Monkey 3:',
  '  Starting items: 80, 64, 92, 93, 64, 56',
  '  Operation: new = old + 5',
  '  Test: divisible by 7',
  '    If true: throw to monkey 2',
  '    If false: throw to monkey 0'],
 ['Monkey 4:',
  '  Starting items: 70, 88, 95, 99, 78, 72, 65, 94',
  '  Operation: new = old + 7',
  '  Test: divisible by 2',
  '    If true: throw to monkey 7',
  '    If false: throw to monkey 5'],
 ['Monkey 5:',
  '  Starting item

In [140]:
divisors = [int(row[3].split(' ')[-1]) for row in data]
divisors

[17, 3, 19, 7, 2, 5, 11, 13]

In [141]:
mon_lcm = math.lcm(*divisors)
mon_lcm

9699690

In [142]:
from helpers import npNicePrint
from functools import partial
import operator

class monkey():
    def __init__(self, items, opp_variables, test_val, throw_loc, mon_lcm):
        self.items = items
        self.create_operation(*opp_variables)
        self.test_val = test_val
        self.throw_loc = throw_loc
        self.num_inspected = 0
        self.mon_lcm = mon_lcm
    def create_operation(self, op, val_2):
        ops = {
            '+': operator.add,
            '-': operator.sub,
            '*': operator.mul,
            '/': operator.truediv,
            '//': operator.floordiv,
            '%': operator.mod,
            '**': operator.pow
        }
        def opp(val_1, op, val_2):
            return ops[op](val_1, val_2) % self.mon_lcm
        def opp_2(val_1, op):
            return ops[op](val_1, val_1) % self.mon_lcm
        if val_2 == 'old':
            self.operation = partial(opp_2, op=op)
        else:
            self.operation = partial(opp, op = op, val_2 = val_2)
        
    def turn(self):
        while len(self.items) > 0:
            item = self.items.pop(0)
            item = self.inspect(item)
            #item = item #% self.test_val
            self.throw(item)
            
    def inspect(self, item):
        self.num_inspected += 1
        return self.operation(item)
        
    def throw(self, item):
        if item % self.test_val == 0:
            monkeys[self.throw_loc[0]].items.append(item)
        else:
            monkeys[self.throw_loc[1]].items.append(item) 
    def __str__(self):
        return f"{self.items}"

In [143]:
monkeys = []
for row in data:
    items = list(map(int, row[1].split(': ')[1].split(',')))
    opp_variables = row[2].split(' ')[-2:]
    if opp_variables[1].isnumeric():
        opp_variables[1] = int(opp_variables[1])
    test_val = int(row[3].split(' ')[-1])
    throw_loc = [int(row[4].split(' ')[-1]), int(row[5].split(' ')[-1])]
    monkeys.append(monkey(items, opp_variables, test_val, throw_loc, mon_lcm))
monkeys

In [144]:
def monkey_round():
    for monkey in monkeys:
        monkey.turn()
for x in range(10000):
    monkey_round()

In [145]:
[monkey.num_inspected for monkey in monkeys]

[25869, 161523, 146733, 27041, 155504, 30343, 160567, 160360]

In [146]:
str(monkeys[0]), str(monkeys[1]), str(monkeys[2]), str(monkeys[3])

('[6001396, 9000854]',
 '[7329939, 8856151, 3192119, 8673749, 5201269, 7309463, 6465433, 3845827, 3845827, 5500777, 6522149, 5740907, 4787251, 4363881]',
 '[]',
 '[]')

In [147]:
operator.mul(*sorted([monkey.num_inspected for monkey in monkeys])[-2:])

25935263541